In [1]:
import pandas as pd

In [2]:
import numpy as np

## Created functions

In [40]:
def add_zeros(x):
    if x<10:
        return "0"+str(x)
    else:
        return str(x)

In [41]:
def dhont(df_o):
    cols=list(df_o.columns[0:3])
    df_om=pd.melt(df_o, id_vars=cols, value_vars=df_o.columns[3:])
    df_om.rename(columns={"variable":"komitet", "value":"Uzyskane_glosy"}, inplace=True)
    votes_ix= list(df_om.columns).index("Uzyskane_glosy")
    arr_o=df_om.values
    n=arr_o.shape[0]
    m=arr_o.shape[1]

    arr_col=[]
    for j in range(1,22) :  
        if j==1:
                    arr_dhont=np.ones((n,1))
                    arr_dhont=arr_o[:,votes_ix]/j
                    name="Mandat_"+str(j)
                    arr_col.append(name)
        else:
                    arr_temp=np.ones((n,1))
                    arr_temp=arr_o[:,votes_ix]/j
                    name="Mandat_"+str(j)
                    arr_col.append(name)
                    arr_dhont=np.column_stack((arr_dhont, arr_temp))
    df_dhont=pd.DataFrame(arr_dhont, columns=arr_col)
    df_odhont=pd.merge(df_om, df_dhont, left_index=True, right_index=True)
    df_top=pd.melt(df_odhont, id_vars=df_odhont.columns[:4], value_vars=df_odhont.columns[5:])
    df_top.rename(columns={"variable":"Mandat_rank","value":"Glosy_Dhont"}, inplace=True)
    df_top.head()
    df_top.sort_values(by=["Okreg", "Glosy_Dhont"], ascending=False, inplace=True)
    df_top.reset_index(inplace=True)
    df_top["Ones"]=1
    df_top["ranking"]=df_top.groupby("Okreg")["Ones"].cumsum()
    df_top["in_mandaty"]=np.where(df_top.ranking<=(df_top.Mandaty+1),"In","Out")
    df_top=df_top[(df_top.in_mandaty=="In")]
    df_top.head()
    df_top["Mandat_rank"]=df_top.ranking.apply(lambda x: "Mandat_"+add_zeros(x))
    df_top["Mandat_rank"]=np.where(df_top.ranking>df_top.Mandaty, "Runner-up", df_top.Mandat_rank)
    return(df_top)

# Preparing maping of TERYT Codes to Voting Districts in 2019 Parliament Election

In [20]:
name="wyniki_gl_na_kand_po_obwodach_sejm_xlsx\wyniki_gl_na_kand_po_obwodach_sejm_okr_"



for i in range(1,42):
    input_name=name+str(i)+".xlsx"
    if i ==1:
        df_combined=pd.read_excel(input_name, index_col=0).reset_index()
        df_combined=df_combined[["Kod TERYT","Okręg"]]
    else:
        df_load=pd.read_excel(input_name, index_col=0).reset_index()
        df_load=df_load[["Kod TERYT","Okręg"]]
        df_combined=pd.concat([df_combined, df_load])
        
        
    

In [21]:
df_TERYT=df_combined.groupby(["Kod TERYT", "Okręg"]).min().reset_index()

In [22]:
df_TERYT.head()

,Kod TERYT,Okręg
0,20101,1
1,20102,1
2,20103,1
3,20104,1
4,20105,1


# Loading files with community level EU and PL elections from 2019

In [23]:
file_sejm="wyniki_gl_na_listy_po_gminach_sejm.xlsx"
file_eu="Wybory Parlament EU 2019 wyniki_gl_na_listy_po_gminach.xlsx"

In [234]:
df_sejm=pd.read_excel(file_sejm)
df_sejm.head()

,Kod TERYT,Gmina,Powiat,Województwo,Komisja otrzymała kart do głosowania,Liczba wyborców uprawnionych do głosowania,Nie wykorzystano kart do głosowania,"Liczba wyborców, którym wydano karty do głosowania",Liczba wyborców głosujących przez pełnomocnika,Liczba wyborców głosujących na podstawie zaświadczenia o prawie do głosowania,...,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19,KOMITET WYBORCZY AKCJA ZAWIEDZIONYCH EMERYTÓW RENCISTÓW - ZPOW-601-21/19,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19,KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19,KOMITET WYBORCZY PRAWICA - ZPOW-601-20/19,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19,KOMITET WYBORCZY SKUTECZNI PIOTRA LIROYA-MARCA - ZPOW-601-17/19,KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19,KOMITET WYBORCZY WYBORCÓW KOALICJA BEZPARTYJNI I SAMORZĄDOWCY - ZPOW-601-10/19,KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19
0,20101,m. Bolesławiec,bolesławiecki,dolnośląskie,27857,29895,9656,18201,17,96,...,4308,NaN,1109,972,NaN,6445,NaN,3997,1158.0,NaN
1,20102,gm. Bolesławiec,bolesławiecki,dolnośląskie,10224,11171,3907,6317,0,29,...,1197,NaN,472,577,NaN,2716,NaN,930,331.0,NaN
2,20103,gm. Gromadka,bolesławiecki,dolnośląskie,4014,4306,1739,2274,1,8,...,287,NaN,117,154,NaN,1128,NaN,428,106.0,NaN
3,20104,gm. Nowogrodziec,bolesławiecki,dolnośląskie,10637,11683,4937,5700,1,24,...,878,NaN,432,537,NaN,2980,NaN,545,219.0,NaN
4,20105,gm. Osiecznica,bolesławiecki,dolnośląskie,4905,5392,2212,2693,0,36,...,509,NaN,183,216,NaN,1200,NaN,410,108.0,NaN


In [235]:
df_wybory=pd.merge(df_TERYT, df_sejm, left_on="Kod TERYT", right_on="Kod TERYT")

In [236]:
df_wybory.columns

Index(['Kod TERYT', 'Okręg', 'Gmina', 'Powiat', 'Województwo',
       'Komisja otrzymała kart do głosowania',
       'Liczba wyborców uprawnionych do głosowania',
       'Nie wykorzystano kart do głosowania',
       'Liczba wyborców, którym wydano karty do głosowania',
       'Liczba wyborców głosujących przez pełnomocnika',
       'Liczba wyborców głosujących na podstawie zaświadczenia o prawie do głosowania',
       'Liczba wyborców, którym wysłano pakiety wyborcze',
       'Liczba otrzymanych kopert zwrotnych',
       'Liczba kopert zwrotnych, w których nie było oświadczenia o osobistym i tajnym oddaniu głosu',
       'Liczba kopert zwrotnych, w których oświadczenie nie było podpisane',
       'Liczba kopert zwrotnych, w których nie było koperty na kartę do głosowania',
       'Liczba kopert zwrotnych, w których znajdowała się niezaklejona koperta na kartę do głosowania',
       'Liczba kopert na kartę do głosowania wrzuconych do urny',
       'Liczba kart wyjętych z urny',
       '

In [237]:
sejm_list=['Kod TERYT', 'Okręg',
       'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów',
       'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19',
       'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19',
       'KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19',
       'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19',
       'KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19',
       'KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19']

    

In [238]:
sejm_dict={
      'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów':"Głosy ważne",    'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI - ZPOW-601-6/19':"KO", 
           'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ - ZPOW-601-5/19':"KONFEDERACJA",
           'KOMITET WYBORCZY POLSKIE STRONNICTWO LUDOWE - ZPOW-601-19/19':"PSL",
           'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-601-9/19':"PiS",
            'KOMITET WYBORCZY SOJUSZ LEWICY DEMOKRATYCZNEJ - ZPOW-601-1/19':"SLD",
            'KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA - ZPOW-601-15/19':"MN"
          }

In [239]:
df_pl=df_wybory[sejm_list]
df_pl.rename(columns=sejm_dict, inplace=True)
df_o=df_pl.groupby("Okręg").sum()
df_o.drop(columns="Kod TERYT", inplace=True)
df_o.head(10)

,Głosy ważne,KO,KONFEDERACJA,PSL,PiS,SLD,MN
Okręg,,,,,,,
1,432436,108191,25319,31006,183364,71061,0.0
2,283002,90812,15346,20528,114728,34957,0.0
3,654455,214629,48775,42269,226915,100843,0.0
4,459982,142844,32406,41497,167550,69763,0.0
5,452330,119526,28625,49225,182648,67076,0.0
6,565597,109185,40012,51474,313284,44152,0.0
7,401318,59401,23439,47604,238802,27404,0.0
8,437917,136955,31490,50943,150188,68341,0.0
9,415540,148830,27627,18828,136731,83524,0.0


In [240]:
df_o=df_pl.groupby("Okręg").sum()

In [241]:
okreg_map=pd.read_excel("Okreg-mandat-map.xlsx")
okreg_map.head()

,Okreg,Mandaty
0,1,12
1,2,8
2,3,14
3,4,12
4,5,13


In [242]:
df_o=pd.merge(df_o, okreg_map, left_on="Okręg", right_on="Okreg")

In [243]:
df_o=df_o[['Okreg', 'Mandaty','Głosy ważne', 'KO', 'KONFEDERACJA', 'PSL', 'PiS', 'SLD',"MN"]]
cols=list(df_o.columns[0:3])


In [244]:
df_o.head()

,Okreg,Mandaty,Głosy ważne,KO,KONFEDERACJA,PSL,PiS,SLD,MN
0,1,12,432436,108191,25319,31006,183364,71061,0.0
1,2,8,283002,90812,15346,20528,114728,34957,0.0
2,3,14,654455,214629,48775,42269,226915,100843,0.0
3,4,12,459982,142844,32406,41497,167550,69763,0.0
4,5,13,452330,119526,28625,49225,182648,67076,0.0


In [245]:
df_sejm_mandates=dhont(df_o)

In [246]:
df_sejm_mandates.head()

,index,Okreg,Mandaty,Głosy ważne,komitet,Mandat_rank,Glosy_Dhont,Ones,ranking,in_mandaty
0,40,41,12,470529,KO,Mandat_01,168022,1,1,In
1,163,41,12,470529,PiS,Mandat_02,165200,1,2,In
2,286,41,12,470529,KO,Mandat_03,84011,1,3,In
3,409,41,12,470529,PiS,Mandat_04,82600,1,4,In
4,204,41,12,470529,SLD,Mandat_05,71756,1,5,In


# District mandates per Party

In [247]:
glosy_by_okreg=df_sejm_mandates.groupby("Okreg")["Głosy ważne"].max().reset_index()
mandaty_by_komitet=df_sejm_mandates.pivot(index="Okreg", columns="Mandat_rank", values="komitet")
mandaty_by_komitet=pd.merge(mandaty_by_komitet, glosy_by_okreg, left_on="Okreg", right_on="Okreg")

In [248]:
mandaty_by_komitet.head()

,Okreg,Mandat_01,Mandat_02,Mandat_03,Mandat_04,Mandat_05,Mandat_06,Mandat_07,Mandat_08,Mandat_09,...,Mandat_13,Mandat_14,Mandat_15,Mandat_16,Mandat_17,Mandat_18,Mandat_19,Mandat_20,Runner-up,Głosy ważne
0,1,PiS,KO,PiS,SLD,PiS,KO,PiS,PiS,KO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KO,432436
1,2,PiS,KO,PiS,KO,PiS,SLD,KO,PiS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PiS,283002
2,3,PiS,KO,PiS,KO,SLD,PiS,KO,PiS,KO,...,KO,PSL,NaN,NaN,NaN,NaN,NaN,NaN,PiS,654455
3,4,PiS,KO,PiS,KO,SLD,PiS,KO,PiS,PSL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KONFEDERACJA,459982
4,5,PiS,KO,PiS,SLD,PiS,KO,PSL,PiS,KO,...,KO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KONFEDERACJA,452330


## Final count of mandates - Actual

In [249]:
ser_fin=df_sejm_mandates[(df_sejm_mandates.Mandat_rank!="Runner-up")].groupby("komitet").Ones.sum()

MANDATES_FIN=pd.DataFrame(ser_fin).rename(columns={"Ones":"Mandates_Actual"}).sort_values(by="Mandates_Actual", ascending=False).reset_index()
MANDATES_FIN

,komitet,Mandates_Actual
0,PiS,235
1,KO,134
2,SLD,49
3,PSL,30
4,KONFEDERACJA,11
5,MN,1


# District mandates by vote count threshold

In [253]:
mandaty_by_glosy=df_sejm_mandates.pivot(index="Okreg", columns="Mandat_rank", values="Glosy_Dhont")
mandaty_by_glosy=pd.merge(mandaty_by_glosy, glosy_by_okreg, left_on="Okreg", right_on="Okreg")

In [254]:
mandaty_by_glosy.head()

,Okreg,Mandat_01,Mandat_02,Mandat_03,Mandat_04,Mandat_05,Mandat_06,Mandat_07,Mandat_08,Mandat_09,...,Mandat_13,Mandat_14,Mandat_15,Mandat_16,Mandat_17,Mandat_18,Mandat_19,Mandat_20,Runner-up,Głosy ważne
0,1,183364,108191,91682,71061,61121.3,54095.5,45841,36672.8,36063.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27047.8,432436
1,2,114728,90812,57364,45406,38242.7,34957,30270.7,28682,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22945.6,283002
2,3,226915,214629,113458,107314,100843,75638.3,71543,56728.8,53657.2,...,42925.8,42269,NaN,NaN,NaN,NaN,NaN,NaN,37819.2,654455
3,4,167550,142844,83775,71422,69763,55850,47614.7,41887.5,41497,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32406,459982
4,5,182648,119526,91324,67076,60882.7,59763,49225,45662,39842,...,29881.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28625,452330


In [255]:
mandaty_by_glosy.to_excel("mandaty_by_glosy.xlsx")
mandaty_by_komitet.to_excel("mandaty_by_komitet.xlsx")

# Analysis of districts without SLD mandate

In [256]:
df_fin=df_sejm_mandates[(df_sejm_mandates.Mandat_rank!="Runner-up")]
df_fin.drop(columns="index", inplace=True)
#Looking for district where SLD didn't get a mandate
dist_list=df_fin.Okreg.unique()
sld_list=df_fin[(df_fin.komitet=="SLD")].Okreg.unique()


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [257]:
dist_list

array([41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25,
       24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,
        7,  6,  5,  4,  3,  2,  1], dtype=int64)

In [258]:
sld_list

array([41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25,
       24, 23, 21, 20, 19, 16, 13, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,
        1], dtype=int64)

In [259]:
no_sld_list=list(set(dist_list)-set(sld_list))

In [260]:
no_sld_list

[12, 14, 15, 17, 18, 22]

# What would happen if SLD was in coalition with KO in these districts?

In [261]:
df_o2=df_o.copy()
df_o2["no_sld_mandate"]=df_o2.Okreg.apply(lambda x: x in no_sld_list)
df_o.head(15)

,Okreg,Mandaty,Głosy ważne,KO,KONFEDERACJA,PSL,PiS,SLD,MN
0,1,12,432436,108191,25319,31006,183364,71061,0.0
1,2,8,283002,90812,15346,20528,114728,34957,0.0
2,3,14,654455,214629,48775,42269,226915,100843,0.0
3,4,12,459982,142844,32406,41497,167550,69763,0.0
4,5,13,452330,119526,28625,49225,182648,67076,0.0
5,6,15,565597,109185,40012,51474,313284,44152,0.0
6,7,12,401318,59401,23439,47604,238802,27404,0.0
7,8,12,437917,136955,31490,50943,150188,68341,0.0
8,9,10,415540,148830,27627,18828,136731,83524,0.0
9,10,9,346326,54160,23427,36151,194658,37930,0.0


In [262]:
df_o2["KO"]=np.where(df_o2.no_sld_mandate, df_o2.SLD + df_o2.KO, df_o2.KO)
df_o2["SLD"]=np.where(df_o2.no_sld_mandate, 0, df_o2.SLD)
df_o2.head(15)

,Okreg,Mandaty,Głosy ważne,KO,KONFEDERACJA,PSL,PiS,SLD,MN,no_sld_mandate
0,1,12,432436,108191,25319,31006,183364,71061,0.0,False
1,2,8,283002,90812,15346,20528,114728,34957,0.0,False
2,3,14,654455,214629,48775,42269,226915,100843,0.0,False
3,4,12,459982,142844,32406,41497,167550,69763,0.0,False
4,5,13,452330,119526,28625,49225,182648,67076,0.0,False
5,6,15,565597,109185,40012,51474,313284,44152,0.0,False
6,7,12,401318,59401,23439,47604,238802,27404,0.0,False
7,8,12,437917,136955,31490,50943,150188,68341,0.0,False
8,9,10,415540,148830,27627,18828,136731,83524,0.0,False
9,10,9,346326,54160,23427,36151,194658,37930,0.0,False


In [263]:
df_sejm_mandates_mod1=dhont(df_o2)

## Final mandates split for joined KO and SLD in 6 districts where SLD had no mandate

In [264]:
ser_trim_sld=df_sejm_mandates_mod1[(df_sejm_mandates_mod1.Mandat_rank!="Runner-up")].groupby("komitet").Ones.sum()
MANDATES_MOD2=pd.DataFrame(ser_trim_sld).rename(columns={"Ones":"Mandates"}).sort_values(by="Mandates", ascending=False).reset_index()
MANDATES_MOD2

,komitet,Mandates
0,PiS,232
1,KO,138
2,SLD,49
3,PSL,29
4,KONFEDERACJA,11
5,MN,1


# Was this predictable based on EU Parliament elections in May?

In [267]:
df_wybory_eu=pd.read_excel(file_eu)
df_wybory_eu.head()

,TERYT,Jednostka terytorialna,Komisja otrzymała kart do głosowania,Liczba wyborców uprawnionych do głosowania,w tym umieszczonych w części A spisu wyborców,w tym umieszczonych w części B spisu wyborców,Nie wykorzystano kart do głosowania,"Liczba wyborców, którym wydano karty do głosowania",w tym w części A spisu wyborców,w tym w części B spisu wyborców,...,KOALICYJNY KOMITET WYBORCZY KOALICJA EUROPEJSKA PO PSL SLD .N ZIELONI - ZPOW-603-7/19,"KOALICYJNY KOMITET WYBORCZY LEWICA RAZEM - PARTIA RAZEM, UNIA PRACY, RSS - ZPOW-603-4/19",KOALICYJNY KOMITET WYBORCZY POLEXIT - KOALICJA - ZPOW-603-14/19,KOMITET WYBORCZY JEDNOŚĆ NARODU - ZPOW-603-11/19,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-603-5/19,KOMITET WYBORCZY RUCH PRAWDZIWA EUROPA - EUROPA CHRISTI - ZPOW-603-9/19,KOMITET WYBORCZY WIOSNA ROBERTA BIEDRONIA - ZPOW-603-8/19,KOMITET WYBORCZY WYBORCÓW KONFEDERACJA KORWIN BRAUN LIROY NARODOWCY - ZPOW-603-3/19,KOMITET WYBORCZY WYBORCÓW KUKIZ'15 - ZPOW-603-1/19,KOMITET WYBORCZY WYBORCÓW POLSKA FAIR PLAY BEZPARTYJNI GWIAZDOWSKI - ZPOW-603-6/19
0,20101,m. Bolesławiec,23581,30273,30273,0,9948,13631,13631,0,...,6408,154,NaN,NaN,4875,NaN,892,725,494,NaN
1,20102,gm. Bolesławiec,8499,11166,11166,0,4102,4397,4397,0,...,1536,46,NaN,NaN,2060,NaN,237,271,216,NaN
2,20103,gm. Gromadka,3394,4376,4374,2,1782,1612,1611,1,...,454,17,NaN,NaN,881,NaN,76,91,61,NaN
3,20104,gm. Nowogrodziec,8772,11854,11854,0,4849,3923,3923,0,...,980,53,NaN,NaN,2236,NaN,197,225,171,NaN
4,20105,gm. Osiecznica,4000,5369,5369,0,2144,1856,1856,0,...,614,44,NaN,NaN,797,NaN,131,135,99,NaN


In [268]:
df_wybory_eu=pd.merge(df_TERYT, df_wybory_eu, left_on="Kod TERYT", right_on="TERYT")

In [269]:
eu_list=['Okręg', 
       'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów',
       'KOALICYJNY KOMITET WYBORCZY KOALICJA EUROPEJSKA PO PSL SLD .N ZIELONI - ZPOW-603-7/19',
       'KOALICYJNY KOMITET WYBORCZY LEWICA RAZEM - PARTIA RAZEM, UNIA PRACY, RSS - ZPOW-603-4/19',
       'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-603-5/19',
       'KOMITET WYBORCZY WIOSNA ROBERTA BIEDRONIA - ZPOW-603-8/19',
       'KOMITET WYBORCZY WYBORCÓW KONFEDERACJA KORWIN BRAUN LIROY NARODOWCY - ZPOW-603-3/19',
       "KOMITET WYBORCZY WYBORCÓW KUKIZ'15 - ZPOW-603-1/19",
       'KOMITET WYBORCZY WYBORCÓW POLSKA FAIR PLAY BEZPARTYJNI GWIAZDOWSKI - ZPOW-603-6/19']

In [270]:
eu_dict={
         'Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów':"Głosy ważne",  
         'KOALICYJNY KOMITET WYBORCZY KOALICJA EUROPEJSKA PO PSL SLD .N ZIELONI - ZPOW-603-7/19':"KO",
         'KOALICYJNY KOMITET WYBORCZY LEWICA RAZEM - PARTIA RAZEM, UNIA PRACY, RSS - ZPOW-603-4/19':"LEWICA_RAZEM",
         'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ - ZPOW-603-5/19':"PiS",
         'KOMITET WYBORCZY WIOSNA ROBERTA BIEDRONIA - ZPOW-603-8/19':"WIOSNA",
         'KOMITET WYBORCZY WYBORCÓW KONFEDERACJA KORWIN BRAUN LIROY NARODOWCY - ZPOW-603-3/19':"NARODOWCY",
         "KOMITET WYBORCZY WYBORCÓW KUKIZ'15 - ZPOW-603-1/19":"KUKIZ",
         'KOMITET WYBORCZY WYBORCÓW POLSKA FAIR PLAY BEZPARTYJNI GWIAZDOWSKI - ZPOW-603-6/19':"FAIR_PLAY"
    
    
    
          }

In [271]:
df_eu=df_wybory_eu[eu_list]
df_eu.rename(columns=eu_dict, inplace=True)
df_o_eu=df_eu.groupby("Okręg").sum()
df_o_eu.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Głosy ważne,KO,LEWICA_RAZEM,PiS,WIOSNA,NARODOWCY,KUKIZ,FAIR_PLAY
Okręg,,,,,,,,
1,316858,128710,4333,135175,20346,15037,13257,0.0
2,212517,91498,2810,85577,14994,9535,8103,0.0
3,484072,221730,6794,171394,37997,27092,19065,0.0
4,338447,164141,3521,126766,19721,13496,10802,0.0
5,325185,141221,4012,133642,19691,11727,14892,0.0
6,432342,129023,3352,246092,14440,20454,15667,3314.0
7,307527,79369,1950,190047,8252,12252,13900,1757.0
8,316422,151214,3690,116302,22802,12313,10101,0.0
9,320940,165937,6543,104533,21266,12991,9670,0.0


## Assumption - LEWICA RAZEM & WIOSNA, KONFEDERACJA & FAIR PLAY as combined entities

In [272]:
df_o_eu["LEWICA"]=df_o_eu.LEWICA_RAZEM+df_o_eu.WIOSNA
df_o_eu["KONFEDERACJA"]=df_o_eu.NARODOWCY+df_o_eu.FAIR_PLAY
df_o_eu["KONFEDERACJA"]=df_o_eu["KONFEDERACJA"].apply(lambda x:int(x))

In [273]:
df_o_eu.head(10)

,Głosy ważne,KO,LEWICA_RAZEM,PiS,WIOSNA,NARODOWCY,KUKIZ,FAIR_PLAY,LEWICA,KONFEDERACJA
Okręg,,,,,,,,,,
1,316858,128710,4333,135175,20346,15037,13257,0.0,24679,15037
2,212517,91498,2810,85577,14994,9535,8103,0.0,17804,9535
3,484072,221730,6794,171394,37997,27092,19065,0.0,44791,27092
4,338447,164141,3521,126766,19721,13496,10802,0.0,23242,13496
5,325185,141221,4012,133642,19691,11727,14892,0.0,23703,11727
6,432342,129023,3352,246092,14440,20454,15667,3314.0,17792,23768
7,307527,79369,1950,190047,8252,12252,13900,1757.0,10202,14009
8,316422,151214,3690,116302,22802,12313,10101,0.0,26492,12313
9,320940,165937,6543,104533,21266,12991,9670,0.0,27809,12991


In [274]:
adj_columns=['Głosy ważne', 'KO', 'PiS',  'KONFEDERACJA',
       'KUKIZ','LEWICA']
df_o_eu=df_o_eu[adj_columns]


In [275]:
df_o_eu=pd.merge(df_o_eu, okreg_map, left_on="Okręg", right_on="Okreg")

In [276]:
df_o_eu=df_o_eu[['Okreg', 'Mandaty','Głosy ważne','KO', 'PiS', 'KONFEDERACJA', 'KUKIZ', 'LEWICA',]]

In [277]:
df_o_eu.head(10)

,Okreg,Mandaty,Głosy ważne,KO,PiS,KONFEDERACJA,KUKIZ,LEWICA
0,1,12,316858,128710,135175,15037,13257,24679
1,2,8,212517,91498,85577,9535,8103,17804
2,3,14,484072,221730,171394,27092,19065,44791
3,4,12,338447,164141,126766,13496,10802,23242
4,5,13,325185,141221,133642,11727,14892,23703
5,6,15,432342,129023,246092,23768,15667,17792
6,7,12,307527,79369,190047,14009,13900,10202
7,8,12,316422,151214,116302,12313,10101,26492
8,9,10,320940,165937,104533,12991,9670,27809
9,10,9,254315,68797,147853,11481,9971,16213


In [278]:
df_eu_mandates=dhont(df_o_eu)

In [279]:
df_eu_mandates.head()

,index,Okreg,Mandaty,Głosy ważne,komitet,Mandat_rank,Glosy_Dhont,Ones,ranking,in_mandaty
0,40,41,12,358321,KO,Mandat_01,174025,1,1,In
1,81,41,12,358321,PiS,Mandat_02,130551,1,2,In
2,245,41,12,358321,KO,Mandat_03,87012.5,1,3,In
3,286,41,12,358321,PiS,Mandat_04,65275.5,1,4,In
4,450,41,12,358321,KO,Mandat_05,58008.3,1,5,In


In [280]:
ser_eu=df_eu_mandates[(df_eu_mandates.Mandat_rank!="Runner-up")].groupby("komitet").Ones.sum()
MANDATES_EU=pd.DataFrame(ser_eu).rename(columns={"Ones":"Mandates"}).sort_values(by="Mandates", ascending=False).reset_index()
MANDATES_EU

,komitet,Mandates
0,PiS,248
1,KO,193
2,LEWICA,17
3,KONFEDERACJA,2


In [307]:
df_temp=df_eu_mandates[(df_eu_mandates.Mandat_rank!="Runner-up")]
df_temp=pd.DataFrame(df_temp.groupby(["Okreg", "komitet"]).Ones.sum()).rename(columns={"Ones":"Mandaty"}).reset_index()
df_temp=df_temp.pivot(index="Okreg", columns="komitet", values="Mandaty")
df_temp=pd.merge(df_temp, okreg_map, left_on="Okreg", right_on="Okreg")
df_temp["Mandates_wo_PIS"]=df_temp.Mandaty-df_temp.PiS
df_temp["PiS_share"]=df_temp["PiS"]/df_temp.Mandaty

In [308]:
okreg_map.head()

,Okreg,Mandaty
0,1,12
1,2,8
2,3,14
3,4,12
4,5,13


In [309]:
df_temp.sort_values(by="PiS_share", ascending=True)

,Okreg,KO,KONFEDERACJA,LEWICA,PiS,Mandaty,Mandates_wo_PIS,PiS_share
18,19,11.0,1.0,2.0,6.0,20,14.0,0.300000
38,39,6.0,NaN,1.0,3.0,10,7.0,0.300000
8,9,6.0,NaN,1.0,3.0,10,7.0,0.300000
24,25,8.0,NaN,NaN,4.0,12,8.0,0.333333
34,35,5.0,NaN,1.0,4.0,10,6.0,0.400000
7,8,6.0,NaN,1.0,5.0,12,7.0,0.416667
3,4,7.0,NaN,NaN,5.0,12,7.0,0.416667
30,31,6.0,NaN,1.0,5.0,12,7.0,0.416667
40,41,6.0,NaN,1.0,5.0,12,7.0,0.416667
25,26,7.0,NaN,1.0,6.0,14,8.0,0.428571


In [322]:
coalition_district_list=df_temp[(df_temp.PiS_share>=0.3)].Okreg.unique()

In [323]:
coalition_district_list.shape

(41,)

# What if analysis for Districts with PiS with >30% mandates

In [326]:
df_o.head()

,Okreg,Mandaty,Głosy ważne,KO,KONFEDERACJA,PSL,PiS,SLD,MN
0,1,12,432436,108191,25319,31006,183364,71061,0.0
1,2,8,283002,90812,15346,20528,114728,34957,0.0
2,3,14,654455,214629,48775,42269,226915,100843,0.0
3,4,12,459982,142844,32406,41497,167550,69763,0.0
4,5,13,452330,119526,28625,49225,182648,67076,0.0


In [327]:
df_o_mod=df_o.copy()
df_o_mod["coalition_district"]=df_o_mod.Okreg.apply(lambda x: x in coalition_district_list)
df_o_mod.head(15)

,Okreg,Mandaty,Głosy ważne,KO,KONFEDERACJA,PSL,PiS,SLD,MN,coalition_district
0,1,12,432436,108191,25319,31006,183364,71061,0.0,True
1,2,8,283002,90812,15346,20528,114728,34957,0.0,True
2,3,14,654455,214629,48775,42269,226915,100843,0.0,True
3,4,12,459982,142844,32406,41497,167550,69763,0.0,True
4,5,13,452330,119526,28625,49225,182648,67076,0.0,True
5,6,15,565597,109185,40012,51474,313284,44152,0.0,True
6,7,12,401318,59401,23439,47604,238802,27404,0.0,True
7,8,12,437917,136955,31490,50943,150188,68341,0.0,True
8,9,10,415540,148830,27627,18828,136731,83524,0.0,True
9,10,9,346326,54160,23427,36151,194658,37930,0.0,True


In [328]:
df_o_mod["KO"]=np.where(df_o_mod.coalition_district, df_o_mod.SLD + df_o_mod.KO, df_o_mod.KO)
df_o_mod["SLD"]=np.where(df_o_mod.coalition_district, 0, df_o_mod.SLD)
df_o_mod.head(15)

,Okreg,Mandaty,Głosy ważne,KO,KONFEDERACJA,PSL,PiS,SLD,MN,coalition_district
0,1,12,432436,179252,25319,31006,183364,0,0.0,True
1,2,8,283002,125769,15346,20528,114728,0,0.0,True
2,3,14,654455,315472,48775,42269,226915,0,0.0,True
3,4,12,459982,212607,32406,41497,167550,0,0.0,True
4,5,13,452330,186602,28625,49225,182648,0,0.0,True
5,6,15,565597,153337,40012,51474,313284,0,0.0,True
6,7,12,401318,86805,23439,47604,238802,0,0.0,True
7,8,12,437917,205296,31490,50943,150188,0,0.0,True
8,9,10,415540,232354,27627,18828,136731,0,0.0,True
9,10,9,346326,92090,23427,36151,194658,0,0.0,True


In [329]:
df_sejm_mandates_mod=dhont(df_o_mod)

In [330]:
ser_fin=df_sejm_mandates_mod[(df_sejm_mandates_mod.Mandat_rank!="Runner-up")].groupby("komitet").Ones.sum()

MANDATES_MOD3=pd.DataFrame(ser_fin).rename(columns={"Ones":"Mandates"}).sort_values(by="Mandates", ascending=False).reset_index()
MANDATES_MOD3

,komitet,Mandates
0,PiS,229
1,KO,191
2,PSL,28
3,KONFEDERACJA,11
4,MN,1


In [337]:
df_comparison=MANDATES_FIN.copy()

In [338]:
df_comparison

,komitet,Mandates_Actual
0,PiS,235
1,KO,134
2,SLD,49
3,PSL,30
4,KONFEDERACJA,11
5,MN,1


In [339]:
df_comparison["Mandates EU"]=MANDATES_EU.Mandates
df_comparison["Mandates_coalition_in_6_dist"]=MANDATES_MOD2.Mandates
df_comparison["Mandates_coalition_in_41_dist"]=MANDATES_MOD3.Mandates

In [340]:
df_comparison

,komitet,Mandates_Actual,Mandates EU,Mandates_coalition_in_6_dist,Mandates_coalition_in_41_dist
0,PiS,235,248.0,232,229.0
1,KO,134,193.0,138,191.0
2,SLD,49,17.0,49,28.0
3,PSL,30,2.0,29,11.0
4,KONFEDERACJA,11,NaN,11,1.0
5,MN,1,NaN,1,NaN
